### This generates new songs based off of the guitar licks that were intaken

In [24]:
%matplotlib inline

import mido
from mido import Message, MidiFile, MidiTrack
import numpy as np
from math import ceil
import keras
import random
import music21
import glob
import os
import json

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.optimizers import RMSprop
import numpy

import pickle

In [2]:
# load json and create model
with open('model.json', 'r') as json_file:
    loaded_model_json = json_file.read()

model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Loaded model from disk


In [8]:
with open('X.p') as f:
    X = pickle.load(f)

In [15]:
ix = random.randint(0, X.shape[0])
psg_seed = X[ix,:,:]

In [3]:
# if a continuous script
tsteps = 100

start_ix = random.randint(0, steps-maxlen - 1)
psg_seed = music_mat[start_ix:start_ix+maxlen,:]
x = psg_seed[None,:,:]

NameError: name 'steps' is not defined

In [18]:
ls *.json

model.json


In [19]:

with open('model.json','r') as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('model.h5')

In [25]:
optimizer = RMSprop(lr=0.01)
loaded_model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [28]:
model.predict(psg_seed[None, :])

InternalError: Blas SGEMM launch failed : a.shape=(50, 48), b.shape=(48, 128), m=50, n=128, k=48
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape, lstm_1_W_i/read)]]
	 [[Node: Softmax/_131 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_219_Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'MatMul', defined at:
  File "/home/alex/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/alex/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/alex/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/alex/.local/lib/python2.7/site-packages/tr

In [16]:
def new_song_mat(model, psg_seed, tsteps=100):
    """ turns a passage seed to a song with #tsteps
    """
#     start_ix = random.randint(0, steps-maxlen - 1)
#     psg_seed = music_mat[start_ix:start_ix+maxlen,:]
#     x = psg_seed[None,:,:]

    new_song = []
    new_song.append(x)

    for _ in range(tsteps):

        preds = model.predict(x, verbose=0)[0]

        # this is a timestep slice
        new_note = np.zeros(nnotes)
        new_note[preds.argmax()] = 1
        new_note = new_note[None, None,:]

        # redo the seed psg
        psg = np.concatenate([x,new_note], axis=1)
        x = psg[:, 1:,:]

        new_song.append(new_note)

    new_song = np.concatenate(new_song, axis=1)
    new_song = np.squeeze(new_song)
    return new_song

SyntaxError: invalid syntax (<ipython-input-16-70b5f2f28935>, line 7)

In [15]:
def notelist_to_track(new_song):   
    """ Converts a notelist to a miditrack.
    Assumes that adjacent notes are held
    """
    track = MidiTrack()
    nsteps = new_song.shape[0]
    last_event = 0
    for ix in range(nsteps):
        step_slice = new_song[ix,:]

        diff = [ix for ix,(x,y) in enumerate(zip(step_slice, active_notes)) if x!=y]
        if len(diff) == 0:
            last_event += 1
            continue

        # this means there is a difference
        for note_ix in diff:
            # off to on
            note = note_ix+min(note_range)
            if active_notes[note_ix] == 0:
                track.append(Message('note_on', note=note, velocity=127, time=int(tick_step*last_event) ))
                active_notes[note_ix] = 1
            else:
                track.append(Message('note_off', note=note, velocity=127, time=int(tick_step*last_event)))
                active_notes[note_ix] = 0
            last_event = 0
    return track

In [ ]:
mid = MidiFile()
mid.tracks.append(track)
